In [1]:
from funcs import load_matrices
import numpy as np
import matplotlib.pyplot as plt
import os

dirs = [n for n in os.listdir() if (os.path.isdir(n) and ((n[-2:]=='_p') or (n[-2:]=='_s')))]
for dir in dirs:
    smats_file = np.load(f'{dir}/Smatrix.npz', allow_pickle=True)['arr_0'][()]
    r, hls, ddrls = load_matrices(dir)
    n = len(hls[-1])
    es = np.array([float(k.split('=')[-1]) for k in smats_file.keys()])
    smats = [smats_file[f'E = {es[ei]}'] for ei in range(len(es))]
    ks = [np.array([np.sqrt((es[ei]-np.real(np.diag(hls[-1])[i]))/4.637) for i in range(n) if es[ei] > np.real(np.diag(hls[-1])[i])]) for ei in range(len(es))]
    sigma_sum = [np.pi*1e-16*np.sum(np.array([(2*li+1)*np.abs(smats[ei][li]-np.eye(len(smats[ei][li])))**2 for li in range(len(smats[ei]))]), axis=0) @ np.diag(ks[ei]**(-2)) for ei in range(len(es))]
    sigmas = np.zeros((len(es), n, n))
    for ei in range(len(es)):
        nt = len(sigma_sum[ei])
        sigmas[ei, :nt, :nt] = sigma_sum[ei]

    headerfinal = 'E, cm-1\t'
    for i in range(n):
        for j in range(n):
            headerfinal += f'Sigma_{i}{j}, cm2\t'

    tofile = np.zeros((len(es), n*n+1))
    tofile[:,0] = es
    for i in range(n):
        for j in range(n):
            tofile[:,i*n+j+1] = sigmas[:,i,j]

    np.savetxt(f'{dir}/sigmas_total_airy.txt', tofile, fmt='%.6e', delimiter='\t', header=headerfinal, comments='')